In [1]:
# Importing required packages
import time
import warnings; warnings.simplefilter('ignore')
program_start_time  = time.process_time()
import os
from autocorrect import Speller
import numpy as np
import pandas as pd
import gensim
import re
import nltk
import spacy
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
stemmer = PorterStemmer()
np.random.seed(2018)
import nltk
nltk.download('stopwords')
from pprint import pprint
nltk.download('wordnet')
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

# Allowing notebook to display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(os.getcwd())

[nltk_data] Downloading package stopwords to C:\Users\Sundararaman
[nltk_data]     Sairam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sundararaman
[nltk_data]     Sairam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


C:\Users\Sundararaman Sairam


In [2]:
#Data Import 
#original 
path=os.getcwd()
input_df=pd.read_csv("C:/Work/NLP_POC/01.Input Data/Bosch/Latest_Mapped_Data_20181013.csv",encoding='Latin-1')
filtered_df=input_df[["Primary Key","Validation status","features_preproc","Translate"]]
del(input_df)

# Removing records with blank Primary key values and only records in validation status that are validated by BCG
filtered_df=filtered_df[~(filtered_df['Primary Key'].isna()) ]
#filtered_df=filtered_df[(filtered_df['Validation status']=="Validated by BCG")]
filtered_df=filtered_df[(filtered_df['Validation status']=="Validated by BCG") | (filtered_df['Validation status']== "Validated by BCG - To be checked") | (filtered_df['Validation status']== "Single BCG L3 mapped - Pending QC") | (filtered_df['Validation status']== " Single BCG L3 Mapped - Pending QC")]

In [3]:
#Inner Join with mapping file get correspoding labels
#Inner Join with mapping file get correspoding labels
filtered_df['Level1']=filtered_df['Primary Key'].apply(lambda x:(x.split('|'))[0])
filtered_df['Level2']=filtered_df['Primary Key'].apply(lambda x:(x.split('|'))[1])
filtered_df['Level3']=filtered_df['Primary Key'].apply(lambda x:(x.split('|'))[2])
filtered_df['Level1']=filtered_df['Level1'].apply(lambda x : (x.strip()).upper())
filtered_df['Level2']=filtered_df['Level2'].apply(lambda x : (x.strip()).upper())
filtered_df['Level3']=filtered_df['Level3'].apply(lambda x : (x.strip()).upper())
filtered_df['Primary Key_new']=filtered_df['Level1']+'|'+ filtered_df['Level2'] + '|' +  filtered_df['Level3']
#Please look at the data once to ensure correct concatenation

#Dropping unnecessary columns
filtered_df=filtered_df.drop(['Primary Key','Validation status','Level1','Level2','Level3','Translate'],axis=1)
#Renaming the column of primary key as Level1 to maintain nomenclature
filtered_df.rename(columns={'Primary Key_new':'Level1'},inplace=True)
filtered_df['originalInvoiceText']=filtered_df['features_preproc']
filtered_df['Invoice Text']=filtered_df['originalInvoiceText']
filtered_df['data_source']='Bosch'
filtered_df=filtered_df.drop(['features_preproc'],axis=1)

# # Removing keyword translate and [] brackets from invoice text
filtered_df['Invoice Text'] = filtered_df['Invoice Text'].str.replace("\[Translation: ","")
filtered_df['Invoice Text'] = filtered_df['Invoice Text'].str.replace("]","")
full_df=filtered_df[['Invoice Text', 'Level1']]

In [4]:
#Function to create sgement percentage dataset for input column
def column_creation(column_name,input_df):	
	dataset_name=pd.DataFrame()
	dataset_name=input_df[column_name].value_counts().rename_axis(column_name).reset_index(name='counts')
	dataset_name['seg_perc']=(dataset_name['counts']/(dataset_name['counts'].sum()))*100

	return(dataset_name)

In [5]:
df=full_df.drop_duplicates(inplace = False)
print(full_df.shape)
print(df.shape)

(823735, 2)
(61780, 2)


In [6]:
#Removing Duplicate records
df=full_df.drop_duplicates(inplace = False)

#df = full_df.copy()
df['originalInvoiceText'] = df[["Invoice Text"]].astype(str)
df[["Invoice Text"]] = df[["Invoice Text"]].astype(str)

#Creating summary statistics, %Level to identify and eiliminate low percentage 
summary=column_creation('Level1',df)

#Joining and getting the summary statistics
df = pd.merge(df,summary, how = 'left', left_on = ['Level1']
                       , right_on = ['Level1'])[['Level1','originalInvoiceText','Invoice Text','seg_perc']]

#Percentage threshold
threshold=0.5
df=df[df['seg_perc']>threshold]

# Creating index which will come into use later while joining
df['index'] = df.reset_index().index

# Reordering data
#df = df[['index','Level1','originalInvoiceText','Invoice Text']]

#Unique classes check
print("unique classes are-",df['Level1'].nunique())

unique classes are- 20


## 1.0 Data Pre-Processing (Basic) starts here

In [7]:
# Remember - the original raw text is retained in the column originalInvoiceText

#Step 1 - Uniform Case Conversion
# Converting text to lower case
df["Invoice Text"] = df["Invoice Text"].str.lower()

#Step 2 Digits Removal
# Removing digits from text since SKIP GRAM can't handle digits and the digits are in the form of dates  -20170908 and dont add any value
def remove_digits(text):
    return "".join([i for i in text if not i.isdigit()])

df["Invoice Text"] = df["Invoice Text"].apply(lambda text:remove_digits(text))

#Step 3 Replace Punctuations and special characters with space so that work demaraction is maintained
# Removing punctuations
PUNCT_TO_REMOVE =string.punctuation
def rem_punctuation(text):
    
    """custom function to remove the punctuation"""  
  
    return  re.sub(r"[!%:*<>=/(){}^'~|\_,.;@#?!&$¡¢£¤¥¦+§¨©ª«¬®¯°±²³´µ¶·¸¹º»¼½¾¿àáâãäåæçèéêëìíîïðñòóôõö×øùúûüýþßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ]+\ *", " ", text)  
df["Invoice Text"] = df["Invoice Text"].apply(lambda text:rem_punctuation(text))
df["Invoice Text"] = df["Invoice Text"].str.replace("-"," ")

#Step 4  - Remove Stopwords (Enhanced list (jan,january etc )
# Removal of stopwords
STOPWORDS = set(stopwords.words('english'))
#Adding the most commonly used relevant stopwords
STOPWORDS=STOPWORDS|{"a", "about", "above", "across", "after", "afterwards", 
                    "again", "all", "almost", "alone", "along", "already", "also",    
                    "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "as", "at", "be", "became", "because", "become","becomes", "becoming", "been", "before", "behind", "being", "beside", "besides", "between", "beyond", "both", "but", "by","can", "cannot", "cant", "could", "couldnt", "de", "describe", "do", "done", "each", "eg", "either", "else", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "find","for","found", "four", "from", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "indeed", "is", "it", "its", "itself", "keep", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next","no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part","perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "she", "should","since", "sincere","so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "take","than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they",
                    "this", "those", "though", "through", "throughout",
                    "thru", "thus", "to", "together", "too", "toward", "towards",
                    "under", "until", "up", "upon", "us","aaacr","aajit","stupid","ass","nan",
                    "very", "was", "we", "well", "were", "what", "whatever", "when",
                    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
                    "wherein", "whereupon", "wherever", "whether", "which", "while", 
                    "who", "whoever", "whom", "whose", "why", "will", "with",
                    "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves",
                     'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','january','february','march','april','may','june','july','august','september','october','november','december',

                    }
def rem_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df["Invoice Text"] = df["Invoice Text"].apply(lambda text:rem_stopwords(text))

#Step 5 - Lemmatization
# Lemmatization
# Lemmatization - Converting all variants of the words to their 'verb' form
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word,"v") for word in text.split()])
df["Invoice Text"] = df["Invoice Text"].apply(lambda text:lemmatize_words(text))

#Step 6 - Remove short words
#Function to remove words with length<=2
def short_words(text):
    return " ".join([word for word in str(text).split() if len(word)>3])
df["Invoice Text"] = df["Invoice Text"].apply(lambda text:short_words(text))

#Step 7 Rare words removal

#Word frequency count after data pre-processing
# Removal of frequent words
cnt = Counter()
for text in df["Invoice Text"].values:
    for word in str(text).split():
        cnt[word]+=1
        
cnt.most_common(10)
# The most common words seem to be quite indicative of the classes. Hence retaining them for now

# Removal of rare words
n_rare_words = 100
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

# The most rare words seem to be junk values and not add any predictive value to the model. Hence removing them from the corpus
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["Invoice Text"] = df["Invoice Text"].apply(lambda text:remove_rarewords(text))

In [8]:
#Writing the pre-processed data out as a csv file after removing blank texts
df_preprocessed = df.copy()

# Removing text with only 1 word

def remove_shortwords(text):
    """custom function to remove the rare words"""
    return len(text.split())

df_preprocessed["length_row"] = df_preprocessed["Invoice Text"].apply(lambda text:remove_shortwords(text))
df_preprocessed = df_preprocessed.loc[df_preprocessed['length_row']>2,['index','Level1','Invoice Text']]

#df_preprocessed.to_csv("C:/Work/NLP_POC/03.Output Data/DFI/Considering only level 1/Prediction Results/Removing 4 most sparse labels 0.5% threshold\preProcessedDataWithoutSparseLabels.csv", index=False)

## 2.0 Splitting into test and train

In [9]:
# split the dataset into training and validation datasets 
modelingDataset = df_preprocessed[['Level1','Invoice Text']].copy()
modelingDataset.reset_index(drop=True)

#Percentage threshold
summary=column_creation('Level1',modelingDataset)

#Joining and getting the summary statistics
modelingDataset = pd.merge(modelingDataset,summary, how = 'left', left_on = ['Level1']
                       , right_on = ['Level1'])
threshold=0.5
modelingDataset=modelingDataset[modelingDataset['seg_perc']>threshold]

X = modelingDataset['Invoice Text']
y = modelingDataset['Level1']

modelingDataset = modelingDataset[['Level1','Invoice Text']]
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X,y,test_size=0.30,random_state=120)

# Creating copy of datasets
train_x_copy = pd.DataFrame(train_x.copy())
valid_x_copy = pd.DataFrame(valid_x.copy())
train_y_copy = pd.DataFrame(train_y.copy())
valid_y_copy = pd.DataFrame(valid_y.copy())

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Corpus size
print("Size of the corpus is",modelingDataset.shape)

Size of the corpus is (38936, 2)


In [10]:
modelingDataset.to_csv("C:/Work/NLP_POC/03.Output Data/DFI/Considering only level 1/Prediction Results/Removing 4 most sparse labels 0.5% threshold\modelingDataset.csv", index=False)

## 3.0 Creation of n gram TF-IDF

In [11]:
# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2))
tfidf_vect_ngram.fit(modelingDataset['Invoice Text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

## 4.0 Performing Modeling iterations

In [12]:
# Writing a generic funtion that can be invoked to run different model iterations
# It accepts the classifier, feature_vector of training data, labels and feature vectors of valid data as inputs
# Using these inputs, the model is trained and accuracy score is computed

def train_model(classifier, feature_vector_train, label, feature_vector_valid,is_neural_net=False):

    # fit the training dataset on the classifier
    clf = classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = clf.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
        
    return clf,predictions,(metrics.accuracy_score(predictions, valid_y)*100),(metrics.balanced_accuracy_score(predictions, valid_y)*100),(metrics.f1_score(predictions, valid_y,average='macro')*100),(metrics.f1_score(predictions, valid_y,average='weighted')*100),(metrics.precision_score(predictions, valid_y,average='macro')*100),(metrics.precision_score(predictions, valid_y,average='weighted')*100),(metrics.recall_score(predictions, valid_y,average='macro')*100),(metrics.recall_score(predictions, valid_y,average='weighted')*100)

### 4.1 Multinomial Naive Bayes

In [13]:
# # Naive Bayes on Word Level TF IDF Vectors
# # Balanced accuracy is average of recall per class
# start = time.process_time()
# clf, predictions,accuracy,balanced_accuracy,macro_f1,weighted_f1,macro_precision,weighted_precision,macro_recall,weighted_recall  = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("NB, WordLevel TF-IDF: Accuracy: ", round(accuracy,2))
# print ("NB, WordLevel TF-IDF: Balanced Accuracy: ", round(balanced_accuracy,2))
# print ("NB, WordLevel TF-IDF: Macro F1 score: ", round(macro_f1,2))
# print ("NB, WordLevel TF-IDF: Weighted F1 score: ", round(weighted_f1,2))
# print ("NB, WordLevel TF-IDF: Macro Precision score: ", round(macro_precision,2))
# print ("NB, WordLevel TF-IDF: Weighted Precision score: ", round(weighted_precision,2))
# print ("NB, WordLevel TF-IDF: Macro Recall score: ", round(macro_recall,2))
# print ("NB, WordLevel TF-IDF: Weighted Recall score: ", round(weighted_recall,2))

# end = time.process_time()
# print("Time taken to run NB model:", end-start)

### 4.1.2 Comparing NB Preditions vs Actuals

In [14]:
# predictions = encoder.inverse_transform(predictions)
# predictions = pd.DataFrame(predictions)
# predictions.rename(columns={0:'Predicted Class'},inplace=True)
# predictions = pd.concat([predictions.reset_index(drop=True), valid_y_copy.reset_index(drop=True),valid_x_copy.reset_index(drop=True)], axis=1)
# predictions.to_csv("C:/Work/NLP_POC/03.Output Data/DFI/Considering only level 1/Prediction Results/Removing 4 most sparse labels 0.5% threshold/predictionResultsWithoutSparseLabels_4MultinominalNBModel.csv", index=False)

### 4.2 Suport Vector Machine

In [15]:
# SVM on Word Level TF IDF Vectors
# Balanced accuracy is average of recall per class
from sklearn.svm import LinearSVC
start = time.process_time()
clf, predictions,accuracy,balanced_accuracy,macro_f1,weighted_f1,macro_precision,weighted_precision,macro_recall,weighted_recall  = train_model(LinearSVC(random_state=0,tol=1e-5), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, WordLevel TF-IDF: Accuracy: ", round(accuracy,2))
print ("SVM, WordLevel TF-IDF: Balanced Accuracy: ", round(balanced_accuracy,2))
print ("SVM, WordLevel TF-IDF: Macro F1 score: ", round(macro_f1,2))
print ("SVM, WordLevel TF-IDF: Weighted F1 score: ", round(weighted_f1,2))
print ("SVM, WordLevel TF-IDF: Macro Precision score: ", round(macro_precision,2))
print ("SVM, WordLevel TF-IDF: Weighted Precision score: ", round(weighted_precision,2))
print ("SVM, WordLevel TF-IDF: Macro Recall score: ", round(macro_recall,2))
print ("SVM, WordLevel TF-IDF: Weighted Recall score: ", round(weighted_recall,2))

end = time.process_time()
print("Time taken to run SVM model:", end-start)

SVM, WordLevel TF-IDF: Accuracy:  88.97
SVM, WordLevel TF-IDF: Balanced Accuracy:  80.48
SVM, WordLevel TF-IDF: Macro F1 score:  77.36
SVM, WordLevel TF-IDF: Weighted F1 score:  89.33
SVM, WordLevel TF-IDF: Macro Precision score:  75.3
SVM, WordLevel TF-IDF: Weighted Precision score:  89.95
SVM, WordLevel TF-IDF: Macro Recall score:  80.48
SVM, WordLevel TF-IDF: Weighted Recall score:  88.97
Time taken to run SVM model: 0.890625


In [16]:
# # Class-wise confusion matrix
# from sklearn.metrics import classification_report
# valid_y = encoder.inverse_transform(valid_y)
# predictions = encoder.inverse_transform(predictions)
# target_names = np.asarray(train_y_copy['Level1'].unique())
# print(classification_report(valid_y,predictions,target_names=target_names))

In [17]:
# predictions = pd.DataFrame(predictions)
# predictions.rename(columns={0:'Predicted Class'},inplace=True)
# predictions = pd.concat([predictions.reset_index(drop=True), valid_y_copy.reset_index(drop=True),valid_x_copy.reset_index(drop=True)], axis=1)
# predictions.to_csv("C:/Work/NLP_POC/03.Output Data/DFI/Considering only level 1/Prediction Results/Removing 4 most sparse labels 0.5% threshold/predictionResultsWithoutSparseLabels_4_SVMModel.csv", index=False)

### 4.3 Random Forest

In [18]:
# # RF on Word Level TF IDF Vectors
# # Balanced accuracy is average of recall per class
# start = time.process_time()
# clf, predictions,accuracy,balanced_accuracy,macro_f1,weighted_f1,macro_precision,weighted_precision,macro_recall,weighted_recall  = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("RF, WordLevel TF-IDF: Accuracy: ", round(accuracy,2))
# print ("RF, WordLevel TF-IDF: Balanced Accuracy: ", round(balanced_accuracy,2))
# print ("RF, WordLevel TF-IDF: Macro F1 score: ", round(macro_f1,2))
# print ("RF, WordLevel TF-IDF: Weighted F1 score: ", round(weighted_f1,2))
# print ("RF, WordLevel TF-IDF: Macro Precision score: ", round(macro_precision,2))
# print ("RF, WordLevel TF-IDF: Weighted Precision score: ", round(weighted_precision,2))
# print ("RF, WordLevel TF-IDF: Macro Recall score: ", round(macro_recall,2))
# print ("RF, WordLevel TF-IDF: Weighted Recall score: ", round(weighted_recall,2))

# end = time.process_time()
# print("Time taken to run RF model:", end-start)

In [19]:
program_end_time = time.process_time()
print("Total module run time:", program_end_time-program_start_time)

Total module run time: 26.765625


# Miscellaneous + Archive

### Creation of SKIP-GRAM word embedding file starts here

In [20]:
# # load the pre-trained word-embedding vectors 
# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn import decomposition, ensemble

# import pandas, xgboost, numpy, textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

# embeddings_index = {}
# for i, line in enumerate(open('C:/Work/NLP_POC/01.Input Data/DFI/wiki-news-300d-1M.vec')):
#     values = line.split()
#     embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# # create a tokenizer 
# token = text.Tokenizer()
# token.fit_on_texts(modelingDataset['Invoice Text'])
# word_index = token.word_index

# # split the dataset into training and validation datasets 
# modelingDataset = df_preprocessed[['Level1','Invoice Text']].copy()
# modelingDataset.reset_index(drop=True)
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(modelingDataset['Invoice Text'], modelingDataset['Level1'],test_size=0.30,random_state=100)

# # Creating copy of datasets
# train_x_copy = pd.DataFrame(train_x.copy())
# valid_x_copy = pd.DataFrame(valid_x.copy())
# train_y_copy = pd.DataFrame(train_y.copy())
# valid_y_copy = pd.DataFrame(valid_y.copy())

# # label encode the target variable 
# encoder = preprocessing.LabelEncoder()
# train_y = encoder.fit_transform(train_y)
# valid_y = encoder.fit_transform(valid_y)

# # convert text to sequence of tokens and pad them to ensure equal length vectors 
# train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=100)
# valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=100)

# # create token-embedding mapping
# embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [21]:
# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn import decomposition, ensemble

# import pandas, xgboost, numpy, textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

### Cross validation results

In [22]:
# # Doing cross validation

# from sklearn.model_selection import KFold,cross_val_score
# kfold = KFold(n_splits=10, random_state=101)

# def cv_model(classifier, feature_vector_train, label, feature_vector_valid,kfold,is_neural_net=False):

#     # fit the training dataset on the classifier
#     #classifier.fit(feature_vector_train, label)
    
#     print(cross_val_score(classifier, feature_vector_train, label, cv=kfold))
    
#     # predict the labels on validation dataset
#     #predictions = classifier.predict(feature_vector_valid)
    
#     #return predictions,(metrics.accuracy_score(predictions, valid_y)*100),(metrics.balanced_accuracy_score(predictions, valid_y)*100),(metrics.f1_score(predictions, valid_y,average='macro')*100),(metrics.f1_score(predictions, valid_y,average='weighted')*100),(metrics.precision_score(predictions, valid_y,average='macro')*100),(metrics.precision_score(predictions, valid_y,average='weighted')*100),(metrics.recall_score(predictions, valid_y,average='macro')*100),(metrics.recall_score(predictions, valid_y,average='weighted')*100)
# cv_model(LinearSVC(random_state=0,tol=1e-5), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram,kfold)

### All obsolete codes below

### Spell correction code below

In [23]:
# # Running spellcheck using Parallel processing
# # Not including this as part of the code right now, because processing time is huge

# import threading
# results = []
# from spellchecker import SpellChecker
# spell = SpellChecker()
# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)

# from multiprocessing.dummy import Pool as ThreadPool
# pool = ThreadPool(4)
# results = pool.map(correct_spellings, df["Invoice Text"])

In [24]:
# # from spellchecker import SpellChecker
# # spell = SpellChecker()
# # def correct_spellings(text):
# #     corrected_text = []
# #     misspelled_words = spell.unknown(text.split())
# #     for word in text.split():
# #         if word in misspelled_words:
# #             corrected_text.append(spell.correction(word))
# #         else:
# #             corrected_text.append(word)
# #     return " ".join(corrected_text)
# # from autocorrect import Speller
# # df["Invoice Text"] = df["Invoice Text"].astype(str)
# # #df["Invoice Text"] = df["Invoice Text"].apply(lambda x: " ".join([Speller(i) for i in x.split()]))
# # df["Invoice Text"] = [' '.join([Speller(i) for i in x.split()]) for x in df["Invoice Text"]]

# star_time = time.process_time()
# from autocorrect import spell
# import concurrent.futures
# import os
# print(os.cpu_count())
# from multiprocessing.dummy import Pool as ThreadPool
# pool = ThreadPool(4)

# from spellchecker import SpellChecker

# spell = SpellChecker()
# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)

# result = pool.map(correct_spellings, df["Invoice Text"])

# #df["Invoice Text"] = df["Invoice Text"].apply(lambda text:correct_spellings(text))

# end_time = time.process_time()

# print("Time taken to run auto correct module:",end_time-start_time)

### Data Pre-Processing (Advanced) starts here

In [25]:
# # # Note: Emoji, emoticons, HTML Tags, URLs may not be present in our data. This step is only a good to have
# # # Removing emojis from text
# def remove_emoji(string):
#     emoji_pattern = re.compile("["
#                            u"\U0001F600-\U0001F64F"  # emoticons
#                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            u"\U00002702-\U000027B0"
#                            u"\U000024C2-\U0001F251"
#                            "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', str(string))

# df["Invoice Text"] = df["Invoice Text"].apply(lambda text:remove_emoji(text))

# # Removal of emoticons
# # :-) is an emoticon
# # 😀 is an emoji

# #Importing libraries
# import re
# from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# def remove_emoticons(text):
#     emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
#     return emoticon_pattern.sub(r'', str(text))

# remove_emoticons("Hello :-)")

# df["Invoice Text"] = df["Invoice Text"].apply(lambda text:remove_emoticons(text))

# # Removal of URLs
# def remove_urls(text):
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     return url_pattern.sub(r'', str(text))

# df["Invoice Text"] = df["Invoice Text"].apply(lambda text:remove_urls(text))

# # Removal of HTML Tags
# from bs4 import BeautifulSoup
# def remove_html(text):
#     return BeautifulSoup(text, "lxml").text

# df["Invoice Text"] = df["Invoice Text"].apply(lambda text:remove_html(text))